In [76]:


import numpy as np
import pandas as pd

import sklearn
from sklearn.decomposition import TruncatedSVD

columns = ['id','id2','task','TaskId','rating']
frame = pd.read_csv('DataRecommenderSystem/DataInterviewVisitor.csv',sep=',', names=columns)
frame.head()

,id,id2,task,TaskId,rating
0,0,1,11010,1,7
1,1,1,10010,2,7
2,2,1,11010,3,8
3,3,1,1000,4,7
4,4,1,10000,5,6


In [77]:
columns2 =['TaskId','TaskName','Task']
tasks = pd.read_csv('DataRecommenderSystem/TasksInterviewVisitor.csv', sep=',', names = columns2, encoding='latin-1')
task_names = tasks[['TaskId','TaskName','Task']]
task_names.head()


,TaskId,TaskName,Task
0,1,Task1,11010
1,2,Task2,10010
2,3,Task3,11010
3,4,Task4,1000
4,5,Task5,10000


In [78]:
combined_task_data = pd.merge(frame, task_names, on='TaskId')
combined_task_data.head()

,id,id2,task,TaskId,rating,TaskName,Task
0,0,1,11010,1,7,Task1,11010
1,6,1,11010,1,9,Task1,11010
2,7,1,11010,1,7,Task1,11010
3,19,1,11010,1,10,Task1,11010
4,22,1,11010,1,4,Task1,11010


In [79]:
combined_task_data.groupby('TaskId')['rating'].count().sort_values(ascending=True).head()

filter_interview = combined_task_data['TaskId']==0
combined_task_data[filter_interview]['TaskName'].unique()

array([], dtype=object)

In [80]:
rating_crosstab = combined_task_data.pivot_table(values='rating', index='id', columns=['TaskName','Task'], fill_value=0.01)
rating_crosstab

TaskName,Task1,Task10,Task11,Task2,Task3,Task4,Task5,Task6,Task7,Task8,Task9
Task,11010,101110,1010,10010,11010,1000,10000,1111110,11110,1110,111010
id,,,,,,,,,,,
0,7.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
1,0.01,0.01,0.01,7.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01
2,0.01,0.01,0.01,0.01,8.00,0.01,0.01,0.01,0.01,0.01,0.01
3,0.01,0.01,0.01,0.01,0.01,7.00,0.01,0.01,0.01,0.01,0.01
4,0.01,0.01,0.01,0.01,0.01,0.01,6.00,0.01,0.01,0.01,0.01
...,...,...,...,...,...,...,...,...,...,...,...
96,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,9.00,0.01,0.01
97,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,9.00,0.01,0.01


In [81]:
rating_crosstab.shape
X = rating_crosstab.T
X.shape

(11, 101)

In [82]:
SVD = TruncatedSVD(n_components=11, random_state=0)

resultant_matrix = SVD.fit_transform(X)

resultant_matrix.shape

(11, 11)

In [83]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape
task_names = rating_crosstab.columns
task_list = list(task_names)
task_list
task = task_list.index(('Task5',10000))
task
corr_1 = corr_mat[task]
corr_1.shape
corr_1

array([-0.09331667, -0.09975869, -0.101541  , -0.09354323, -0.09700873,
       -0.09672392,  1.        , -0.12670962, -0.09624543, -0.10086629,
       -0.05942382])

In [84]:
list(task_names[(corr_1<=1) & (corr_1 > -0.096)])


[('Task1', 11010), ('Task2', 10010), ('Task5', 10000), ('Task9', 111010)]